<a href="https://colab.research.google.com/github/Gusta1311/Sentimentos-ia/blob/main/Sentimentos_Ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk tweepy transformers torch pandas matplotlib

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
modelo_tweeteval = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(modelo_tweeteval)
modelo = AutoModelForSequenceClassification.from_pretrained(modelo_tweeteval)
print("Modelo carregado com sucesso!")


In [ ]:
!pip install scipy


In [ ]:
import nltk
nltk.download('stopwords')  # Faz o download das stopwords corretamente


In [ ]:
from nltk.corpus import stopwords
stopwords_pt = set(stopwords.words('portuguese'))
print(stopwords_pt)  # Deve exibir a lista de palavras comuns do português


In [ ]:
import nltk
nltk.download('stopwords', download_dir='/usr/local/nltk_data')

# Configurar o NLTK para usar o caminho correto
nltk.data.path.append('/usr/local/nltk_data')


In [ ]:
import re
import tweepy #pegar os tweets
import torch #para ia
import nltk #para identificação das palavras e separa e excluir palavras inuteis
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.corpus import stopwords
from scipy.special import softmax

#baixar stopwords do nltk
nltk.download('stopwords')
stopwords_pt = set (stopwords.words('portuguese'))

#Configurar chaves de API do twitter
consumer_key = 'jMt5BARwMdBjXHxW7pGKYlu2t'
consumer_secret = '0yp9IanVEvqrYpYH6DLRgHjqaVcykcFYhYKmVeJFnbWmMOak0S'
access_token = '1315486843304054785-bjlRmNUNsnKdR9QTZjL7svY26A6Y7c'
access_token_secret = 'ZAgVwdiKk7Ewqbk0ebDOncqOLaU9OLU1UQiiXXKJZZOoA'

#Autenticação do twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

#modelo pré-treinado do tweet eval
modelo_tweeteval = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(modelo_tweeteval)
modelo = AutoModelForSequenceClassification.from_pretrained(modelo_tweeteval)
def limpar_texto(texto):
    texto = texto.lower()  # Converter para minúsculas
    texto = re.sub(r"http\S+", "", texto)  # Remover links
    texto = re.sub(r"@\w+", "", texto)  # Remover menções
    texto = re.sub(r"[^a-zA-Zá-úÁ-Ú ]", "", texto)  # Remover caracteres especiais
    palavras = texto.split()
    palavras = [p for p in palavras if p not in stopwords_pt]  # Remover stopwords
    return " ".join(palavras)

#Função para classificar sentimento
def classificar_sentimento(texto):
    tokens = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        resultado = modelo(**tokens)

    #Convrter logits para probabilidade
    pontuacoes = resultado.logits[0].numpy()
    probabilidades = softmax(pontuacoes)

    categorias = ["Negativo", "Neutro", "Positivo"]
    indice = probabilidades.argmax()

    return categorias[indice], probabilidades

 #Escolher entre manual ou analise de tweets
print("Ecolha uma opção:")
print("1 - Digitar frases manualmente")
print("2 - Analisar tweets de um perfil")
print("3 - Analisar um tweet sobre um assunto")

opcao = input ("Digite o numero da opcao desejada: ")

dados = []

if opcao == "1":
    while True:
        frase = input("\nDigite uma frase (ou 'sair' para encerrar): ")
        if frase.lower() == "sair":
            break
        sentimento, probabilidades = classificar_sentimento(frase)
        print(f"Sentimento: {sentimento} | Probabilidades: {probabilidades}\n")
        dados.append({"Texto": frase, "Sentimento": sentimento, "Probabilidades": probabilidades})
elif opcao == "2":
    perfil = input("\nDigite o @ do perfil do Twitter (sem o '@'): ")
    tweets = api.user_timeline(screen_name=perfil, count=10, tweet_mode="extended")

    for tweet in tweets:
        texto_limpo = limpar_texto(tweet.full_text)
        sentimento, probabilidades = classificar_sentimento(texto_limpo)
        dados.append({"Texto": tweet.full_text, "Sentimento": sentimento, "Probabilidades": probabilidades})

elif opcao == "3":
    termo_pesquisa = input("\nDigite um termo para buscar no Twitter: ")
    tweets = api.search_tweets(q=termo_pesquisa, lang="pt", count=10)

    for tweet in tweets:
        texto_limpo = limpar_texto(tweet.text)
        sentimento, probabilidades = classificar_sentimento(texto_limpo)
        dados.append({"Texto": tweet.text, "Sentimento": sentimento, "Probabilidades": probabilidades})

else:
    print("Opção inválida. Saindo do programa.")
    exit()

#Criar DataFrame
df = pd.DataFrame(dados)

#Exibir frases/tweets analisados
print(df)

#Cria gráfico de barras se houver dados
if not df.empty:
  contagem_sentimentos = df["Sentimento"].value_counts()

  plt.figure(figsize=(6,4))
  contagem_sentimentos.plot(kind="bar", color=["red", "gray", "green"])
  plt.title("Análise de Sentimentos")
  plt.xlabel("Sentimento")
  plt.ylabel("Quantidade")
  plt.xticks(rotation=0)
  plt.show()




